In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
np.set_printoptions(suppress=True)
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import datetime

### Pre-processing

In [ ]:
# specify the data file name and url
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00240/'
datafile = url + 'UCI%20HAR%20Dataset.zip'
# download the zip file from the web server using curl
!curl $datafile --output UCI_HAR_Dataset.zip
# unzip the file
!unzip -qq UCI_HAR_Dataset.zip
# change the directory name to remove spaces
!mv -f UCI\ HAR\ Dataset UCI_HAR_DATASET


In [ ]:
# load the features and labels (subtract 1 as the labels aren't indexed from 0)
ytest = np.loadtxt('UCI_HAR_DATASET/test/y_test.txt')-1
ytrain = np.loadtxt('UCI_HAR_DATASET/train/y_train.txt')-1
# load the x,y,z body accelerations test data
xx=np.loadtxt('UCI_HAR_DATASET/test/Inertial Signals/body_acc_x_test.txt')
yy=np.loadtxt('UCI_HAR_DATASET/test/Inertial Signals/body_acc_y_test.txt')
zz=np.loadtxt('UCI_HAR_DATASET/test/Inertial Signals/body_acc_z_test.txt')
# concatenate the arrays along the last dimension
xtest = np.concatenate((xx[:,:,None],yy[:,:,None],zz[:,:,None]),axis=2)
# (using None here adds an extra dimension of size 1 to the end of the array)
# follow the same approach for the train data
xx=np.loadtxt('UCI_HAR_DATASET/train/Inertial Signals/body_acc_x_train.txt')
yy=np.loadtxt('UCI_HAR_DATASET/train/Inertial Signals/body_acc_y_train.txt')
zz=np.loadtxt('UCI_HAR_DATASET/train/Inertial Signals/body_acc_z_train.txt')
xtrain = np.concatenate((xx[:,:,None],yy[:,:,None],zz[:,:,None]),axis=2)


# Part 1.

In [ ]:
# Splitting training data into train and validation sets
x_train, x_val, y_train, y_val = train_test_split(
    xtrain, ytrain, test_size = 0.2, random_state=42)

# Re-shaping training data tensor
x_train_flat = tf.cast(tf.reshape(x_train, (-1,128*3)), tf.float32)
x_val_flat = tf.cast(tf.reshape(x_val, (-1,128*3)), tf.float32)
x_test_flat = tf.cast(tf.reshape(xtest, (-1,128*3)), tf.float32)

# Changing labels to one-hot encoding
y_train1h = tf.cast(tf.keras.utils.to_categorical(y_train), tf.float32)
y_val1h = tf.cast(tf.keras.utils.to_categorical(y_val), tf.float32)
y_test1h = tf.cast(tf.keras.utils.to_categorical(ytest), tf.float32)

# Checking shape of data
print(ytest.shape)
print(ytrain.shape)
print(xtest.shape)
print(xtrain.shape)
print(x_train_flat.shape)
print(x_val_flat.shape)
print(y_train1h.shape)
print(y_val1h.shape)

### Setting up simple multinomial regression model

In [ ]:
# predicted probability for each class
def predict_function(x):
    return tf.nn.softmax(tf.matmul(x,W) + b)

# cross entropy loss function
def loss(y_pred,y):
    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)
    return tf.reduce_mean(-tf.reduce_sum(y * tf.math.log(y_pred), axis=[1]))

# Defining training function
def train_function(x, y):
    with tf.GradientTape() as tape:
        predicted = predict_function(x)
        current_loss = loss(predicted, y)
    gradients = tape.gradient(current_loss, [W, b])
    optimizer.apply_gradients(zip(gradients, [W ,b]))
    return current_loss

# Defining accuracy function
def accuracy(x,y):
    y_ = predict_function(x)
    # calculate where the prediction equals the label
    correct = tf.math.equal(tf.math.argmax(y_,axis=-1),tf.math.argmax(y,axis=-1))
    # convert to a float (previously boolean)
    correct = tf.cast(correct,dtype=tf.float32)
    # return the mean to give the overall accuracy
    return tf.math.reduce_mean(correct)

### Running training

In [ ]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
# Create tensorflow graph
W = tf.Variable(tf.zeros([384, 6]))
b = tf.Variable(tf.zeros([6]))

# Set up logging
logdir = 'tflogs'
writer = tf.summary.create_file_writer(logdir)

epochs = 20000
lr = 0.001
optimizer = tf.keras.optimizers.Adam(lr)
loss_history = np.zeros(epochs)

tf.summary.trace_on()

with writer.as_default():
    for epoch in tqdm(range(epochs)):
        training_loss = train_function(x_train_flat, y_train1h)
        train_acc = accuracy(x_train_flat, y_train1h)
        valid_acc = accuracy(x_val_flat, y_val1h)
        tf.summary.scalar('Model1_loss', training_loss, step = epoch)
        tf.summary.scalar('Model1_train_accuracy', train_acc, step = epoch)
        tf.summary.scalar('Model1_valid_accuracy', valid_acc, step = epoch)


In [ ]:
%tensorboard --logdir /content/tflogs

### Looking at prediction accuracy

In [ ]:
# Training accuracy
print(accuracy(x_train_flat, y_train1h))
# Validation accuracy
print(accuracy(x_val_flat, y_val1h))

In [ ]:
# NOTE: THIS HAS BEEN PUT HERE TO SAVE RE-RUNNING THE MODEL LATER
# Model from Part 1 test accuracy
accuracy(x_test_flat, y_test1h)

# Part 2.

### Creating function for training 1-D convolutional network

In [ ]:
# Setting up function for training network with a single 1-D convolutional layer
def network2(kernel, filters, lr, batch_size, epochs):
    # Constructing CNN model
    cnn1 = tf.keras.Sequential()
    cnn1.add(tf.keras.layers.Conv1D(filters = filters, kernel_size = kernel, input_shape = (128, 3)))
    cnn1.add(tf.keras.layers.BatchNormalization())
    cnn1.add(tf.keras.layers.ReLU())
    cnn1.add(tf.keras.layers.GlobalAveragePooling1D())
    cnn1.add(tf.keras.layers.Dense(units = 6, activation = tf.nn.softmax))
    # Set up logging
    logdir = 'tflogs'
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
    # creating callback to save best model
    callbacks = [
        tf.keras.callbacks.ModelCheckpoint("best_model_cnn1", save_best_only=True, monitor="val_accuracy",
                                           mode='max', verbose = 0),
        tf.keras.callbacks.TensorBoard(log_dir=logdir)
    ]
    # Compiling
    cnn1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = lr),
    			loss='categorical_crossentropy', metrics=['accuracy'])
    # Fitting
    cnn1.fit(x_train, y_train1h, epochs = epochs, batch_size = batch_size, verbose = 0,
            validation_data=(x_val, y_val1h), callbacks=callbacks)


### Trying different learning rates

In [ ]:
%tensorboard --logdir /content/tflogs

In [ ]:
# Remove the previous log folder
!rm -rf ./tflogs/
network2(kernel=4, filters=32, lr=0.01, batch_size=32, epochs=2000)
# recording best model configuration
net1_1 = tf.keras.models.load_model("best_model_cnn1")
# Training accuracy
print("Training loss and accuracy:", net1_1.evaluate(x_train, y_train1h))
# Validation accuracy
print("Validation loss and accuracy:", net1_1.evaluate(x_val, y_val1h))

In [ ]:
# Remove the previous log folder
!rm -rf ./tflogs/
network2(kernel=4, filters=32, lr=0.001, batch_size=32, epochs=2000)
# recording best model configuration
net1_2 = tf.keras.models.load_model("best_model_cnn1")
# Training accuracy
print("Training loss and accuracy:", net1_2.evaluate(x_train, y_train1h))
# Validation accuracy
print("Validation loss and accuracy:", net1_2.evaluate(x_val, y_val1h))

In [ ]:
# Remove the previous log folder
!rm -rf ./tflogs/
network2(kernel=4, filters=32, lr=0.0001, batch_size=32, epochs=2000)
# recording best model configuration
net1_3 = tf.keras.models.load_model("best_model_cnn1")
# Training accuracy
print("Training loss and accuracy:", net1_3.evaluate(x_train, y_train1h))
# Validation accuracy
print("Validation loss and accuracy:", net1_3.evaluate(x_val, y_val1h))

### Testing different batch sizes

In [ ]:
%tensorboard --logdir /content/tflogs

In [ ]:
# Remove the previous log folder
!rm -rf ./tflogs/
# Network 2 with batch size 32
network2(kernel=4, filters=32, lr=0.0001, batch_size=32, epochs=2000)
# recording best model configuration
net2_1 = tf.keras.models.load_model("best_model_cnn1")
# Training accuracy
print("Training loss and accuracy:", net2_1.evaluate(x_train, y_train1h))
# Validation accuracy
print("Validation loss and accuracy:", net2_1.evaluate(x_val, y_val1h))

In [ ]:
# Remove the previous log folder
!rm -rf ./tflogs/
# Network 2 with batch size 64
network2(kernel=4, filters=32, lr=0.0001, batch_size=64, epochs=2000)
# recording best model configuration
net2_2 = tf.keras.models.load_model("best_model_cnn1")
# Training accuracy
print("Training loss and accuracy:", net2_2.evaluate(x_train, y_train1h))
# Validation accuracy
print("Validation loss and accuracy:", net2_2.evaluate(x_val, y_val1h))

In [ ]:
# Remove the previous log folder
!rm -rf ./tflogs/
# Network 2 with batch size 128
network2(kernel=4, filters=32, lr=0.0001, batch_size=128, epochs=2000)
# recording best model configuration
net2_3 = tf.keras.models.load_model("best_model_cnn1")
# Training accuracy
print("Training loss and accuracy:", net2_3.evaluate(x_train, y_train1h))
# Validation accuracy
print("Validation loss and accuracy:", net2_3.evaluate(x_val, y_val1h))

### Testing different network parameters

Different kernal sizes with 16 filters

In [ ]:
%tensorboard --logdir /content/tflogs

In [ ]:
# Remove the previous log folder
!rm -rf ./tflogs/
network2(kernel=3, filters=16, lr=0.0001, batch_size=64, epochs=2000)
# recording best model configuration
net2_4 = tf.keras.models.load_model("best_model_cnn1")
# Training accuracy
print("Training loss and accuracy:", net2_4.evaluate(x_train, y_train1h))
# Validation accuracy
print("Validation loss and accuracy:", net2_4.evaluate(x_val, y_val1h))

In [ ]:
# Remove the previous log folder
!rm -rf ./tflogs/
network2(kernel=4, filters=16, lr=0.0001, batch_size=64, epochs=2000)
# recording best model configuration
net2_5 = tf.keras.models.load_model("best_model_cnn1")
# Training accuracy
print("Training loss and accuracy:", net2_5.evaluate(x_train, y_train1h))
# Validation accuracy
print("Validation loss and accuracy:", net2_5.evaluate(x_val, y_val1h))

In [ ]:
# Remove the previous log folder
!rm -rf ./tflogs/
network2(kernel=5, filters=16, lr=0.0001, batch_size=64, epochs=2000)
# recording best model configuration
net2_6 = tf.keras.models.load_model("best_model_cnn1")
# Training accuracy
print("Training loss and accuracy:", net2_6.evaluate(x_train, y_train1h))
# Validation accuracy
print("Validation loss and accuracy:", net2_6.evaluate(x_val, y_val1h))

Different kernal sizes with 32 filters

In [ ]:
%tensorboard --logdir /content/tflogs

In [ ]:
# Remove the previous log folder
!rm -rf ./tflogs/
network2(kernel=3, filters=32, lr=0.0001, batch_size=64, epochs=2000)
# recording best model configuration
net2_7 = tf.keras.models.load_model("best_model_cnn1")
# Training accuracy
print("Training loss and accuracy:", net2_7.evaluate(x_train, y_train1h))
# Validation accuracy
print("Validation loss and accuracy:", net2_7.evaluate(x_val, y_val1h))

In [ ]:
# Remove the previous log folder
!rm -rf ./tflogs/
network2(kernel=4, filters=32, lr=0.0001, batch_size=64, epochs=2000)
# recording best model configuration
net2_8 = tf.keras.models.load_model("best_model_cnn1")
# Training accuracy
print("Training loss and accuracy:", net2_8.evaluate(x_train, y_train1h))
# Validation accuracy
print("Validation loss and accuracy:", net2_8.evaluate(x_val, y_val1h))

In [ ]:
# Remove the previous log folder
!rm -rf ./tflogs/
network2(kernel=5, filters=32, lr=0.0001, batch_size=64, epochs=2000)
# recording best model configuration
net2_9 = tf.keras.models.load_model("best_model_cnn1")
# Training accuracy
print("Training loss and accuracy:", net2_9.evaluate(x_train, y_train1h))
# Validation accuracy
print("Validation loss and accuracy:", net2_9.evaluate(x_val, y_val1h))

Different kernal sizes with 64 filters

In [ ]:
%tensorboard --logdir /content/tflogs

In [ ]:
# Remove the previous log folder
!rm -rf ./tflogs/
network2(kernel=3, filters=64, lr=0.0001, batch_size=64, epochs=2000)
# recording best model configuration
net2_10 = tf.keras.models.load_model("best_model_cnn1")
# Training accuracy
print("Training loss and accuracy:", net2_10.evaluate(x_train, y_train1h))
# Validation accuracy
print("Validation loss and accuracy:", net2_10.evaluate(x_val, y_val1h))

In [ ]:
# Remove the previous log folder
!rm -rf ./tflogs/
network2(kernel=4, filters=64, lr=0.0001, batch_size=64, epochs=2000)
# recording best model configuration
net2_11 = tf.keras.models.load_model("best_model_cnn1")
# Training accuracy
print("Training loss and accuracy:", net2_11.evaluate(x_train, y_train1h))
# Validation accuracy
print("Validation loss and accuracy:", net2_11.evaluate(x_val, y_val1h))

In [ ]:
# NOTE: THIS HAS BEEN PUT HERE TO SAVE RE-RUNNING THE MODEL LATER
# Model from Part 2 test accuracy
print("Test loss and accuracy:", net2_11.evaluate(xtest, y_test1h))

In [ ]:
# Remove the previous log folder
!rm -rf ./tflogs/
network2(kernel=5, filters=64, lr=0.0001, batch_size=64, epochs=2000)
# recording best model configuration
net2_12 = tf.keras.models.load_model("best_model_cnn1")
# Training accuracy
print("Training loss and accuracy:", net2_12.evaluate(x_train, y_train1h))
# Validation accuracy
print("Validation loss and accuracy:", net2_12.evaluate(x_val, y_val1h))

### Trying network with additional layers

In [ ]:
# Trying two convolutional layers. First layer with a smaller kernal and less filters,
# second layer larger kernal more filters. Batch normalization after each.
# Parameters
# Remove the previous log folder
!rm -rf ./tflogs/
lr = 0.0001
batch_size = 64
epochs = 2000
# Constructing CNN model
cnn1 = tf.keras.Sequential()
cnn1.add(tf.keras.layers.Conv1D(filters = 32, kernel_size = 3, input_shape = (128, 3)))
cnn1.add(tf.keras.layers.BatchNormalization())
cnn1.add(tf.keras.layers.Conv1D(filters = 64, kernel_size = 4))
cnn1.add(tf.keras.layers.BatchNormalization())
cnn1.add(tf.keras.layers.ReLU())
cnn1.add(tf.keras.layers.GlobalAveragePooling1D())
cnn1.add(tf.keras.layers.Dense(units = 6, activation = tf.nn.softmax))
# Set up logging
logdir = 'tflogs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
# creating callback to save best model
callbacks = [
    tf.keras.callbacks.ModelCheckpoint("best_model_cnn1", save_best_only=True, monitor="val_accuracy",
                                       mode='max', verbose = 0),
    tf.keras.callbacks.TensorBoard(log_dir=logdir)]
# Compiling
cnn1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = lr),
			loss='categorical_crossentropy', metrics=['accuracy'])
# Fitting
cnn1.fit(x_train, y_train1h, epochs = epochs, batch_size = batch_size, verbose = 0,
        validation_data=(x_val, y_val1h), callbacks=callbacks)
# recording best model configuration
net2_13 = tf.keras.models.load_model("best_model_cnn1")
# Training accuracy
print("Training loss and accuracy:", net2_13.evaluate(x_train, y_train1h))
# Validation accuracy
print("Validation loss and accuracy:", net2_13.evaluate(x_val, y_val1h))

In [ ]:
%tensorboard --logdir /content/tflogs

In [ ]:
# Trying three convolutional layers.
# Remove the previous log folder
!rm -rf ./tflogs/
# Parameters
lr = 0.0001
batch_size = 64
epochs = 2000
# Constructing CNN model
cnn1 = tf.keras.Sequential()
cnn1.add(tf.keras.layers.Conv1D(filters = 16, kernel_size = 3, input_shape = (128, 3)))
cnn1.add(tf.keras.layers.BatchNormalization())
cnn1.add(tf.keras.layers.Conv1D(filters = 32, kernel_size = 4))
cnn1.add(tf.keras.layers.BatchNormalization())
cnn1.add(tf.keras.layers.Conv1D(filters = 64, kernel_size = 5))
cnn1.add(tf.keras.layers.BatchNormalization())
cnn1.add(tf.keras.layers.ReLU())
cnn1.add(tf.keras.layers.GlobalAveragePooling1D())
cnn1.add(tf.keras.layers.Dense(units = 6, activation = tf.nn.softmax))
# Set up logging
logdir = 'tflogs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
# creating callback to save best model
callbacks = [
    tf.keras.callbacks.ModelCheckpoint("best_model_cnn1", save_best_only=True, monitor="val_accuracy",
                                       mode='max', verbose = 0),
    tf.keras.callbacks.TensorBoard(log_dir=logdir)]
# Compiling
cnn1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = lr),
			loss='categorical_crossentropy', metrics=['accuracy'])
# Fitting
cnn1.fit(x_train, y_train1h, epochs = epochs, batch_size = batch_size, verbose = 0,
        validation_data=(x_val, y_val1h), callbacks=callbacks)
# recording best model configuration
net2_14 = tf.keras.models.load_model("best_model_cnn1")
# Training accuracy
print("Training loss and accuracy:", net2_14.evaluate(x_train, y_train1h))
# Validation accuracy
print("Validation loss and accuracy:", net2_14.evaluate(x_val, y_val1h))

# Part 3.

### Load in the rest of the variables

In [ ]:
# load the x,y,z accelerations for the three data streams for each coordinate for test data
xx=np.loadtxt('UCI_HAR_DATASET/test/Inertial Signals/body_acc_x_test.txt')
yy=np.loadtxt('UCI_HAR_DATASET/test/Inertial Signals/body_acc_y_test.txt')
zz=np.loadtxt('UCI_HAR_DATASET/test/Inertial Signals/body_acc_z_test.txt')
xxt=np.loadtxt('UCI_HAR_DATASET/test/Inertial Signals/total_acc_x_test.txt')
yyt=np.loadtxt('UCI_HAR_DATASET/test/Inertial Signals/total_acc_y_test.txt')
zzt=np.loadtxt('UCI_HAR_DATASET/test/Inertial Signals/total_acc_z_test.txt')
xxg=np.loadtxt('UCI_HAR_DATASET/test/Inertial Signals/body_gyro_x_test.txt')
yyg=np.loadtxt('UCI_HAR_DATASET/test/Inertial Signals/body_gyro_y_test.txt')
zzg=np.loadtxt('UCI_HAR_DATASET/test/Inertial Signals/body_gyro_z_test.txt')
# concatenate into a single (n,128,9) array
xtest_all = np.concatenate((xx[:,:,None],yy[:,:,None],zz[:,:,None],
xxt[:,:,None],yyt[:,:,None],zzt[:,:,None],
xxg[:,:,None],yyg[:,:,None],zzg[:,:,None]),axis=2)

# load the x,y,z accelerations for the three data streams for each coordinate for train data
xx=np.loadtxt('UCI_HAR_DATASET/train/Inertial Signals/body_acc_x_train.txt')
yy=np.loadtxt('UCI_HAR_DATASET/train/Inertial Signals/body_acc_y_train.txt')
zz=np.loadtxt('UCI_HAR_DATASET/train/Inertial Signals/body_acc_z_train.txt')
xxt=np.loadtxt('UCI_HAR_DATASET/train/Inertial Signals/total_acc_x_train.txt')
yyt=np.loadtxt('UCI_HAR_DATASET/train/Inertial Signals/total_acc_y_train.txt')
zzt=np.loadtxt('UCI_HAR_DATASET/train/Inertial Signals/total_acc_z_train.txt')
xxg=np.loadtxt('UCI_HAR_DATASET/train/Inertial Signals/body_gyro_x_train.txt')
yyg=np.loadtxt('UCI_HAR_DATASET/train/Inertial Signals/body_gyro_y_train.txt')
zzg=np.loadtxt('UCI_HAR_DATASET/train/Inertial Signals/body_gyro_z_train.txt')
# concatenate into a single (n,128,9) array
xtrain_all = np.concatenate((xx[:,:,None],yy[:,:,None],zz[:,:,None],
xxt[:,:,None],yyt[:,:,None],zzt[:,:,None],
xxg[:,:,None],yyg[:,:,None],zzg[:,:,None]),axis=2)

# Splitting train data
x_train_all, x_val_all = train_test_split(xtrain_all, test_size = 0.2, random_state=42)

### Using best model configuration from Part 2 with all 9 variables

In [ ]:
# Best perfoming configuration from part 2 using all 9 variables
# Remove the previous log folder
!rm -rf ./tflogs/
# Parameters
kernal = 4
filters = 64
lr = 0.0001
batch_size = 64
epochs = 2000

# Constructing CNN model
cnn2 = tf.keras.Sequential()
cnn2.add(tf.keras.layers.Conv1D(filters = filters, kernel_size = kernal, input_shape = (128, 9)))
cnn2.add(tf.keras.layers.BatchNormalization())
cnn2.add(tf.keras.layers.ReLU())
cnn2.add(tf.keras.layers.GlobalAveragePooling1D())
cnn2.add(tf.keras.layers.Dense(units = 6, activation = tf.nn.softmax))

# Set up logging
logdir = 'tflogs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

# creating callback to save best model
callbacks = [
    tf.keras.callbacks.ModelCheckpoint("best_model_cnn2", save_best_only=True, monitor="val_accuracy",
                                       mode='max', verbose = 0),
    tf.keras.callbacks.TensorBoard(log_dir=logdir)
]

# Compiling
cnn2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = lr),
			loss='categorical_crossentropy', metrics=['accuracy'])
# Fitting
cnn2.fit(x_train_all, y_train1h, epochs = epochs, batch_size = batch_size, verbose = 0,
        validation_data=(x_val_all, y_val1h), callbacks=callbacks)
# recording best model configuration
net3_1 = tf.keras.models.load_model("best_model_cnn2")
# Training accuracy
print("Training loss and accuracy:", net3_1.evaluate(x_train_all, y_train1h))
# Validation accuracy
print("Validation loss and accuracy:", net3_1.evaluate(x_val_all, y_val1h))

### Trying model 3 with multiple convolutional layers

In [ ]:
%tensorboard --logdir /content/tflogs

In [ ]:
# Remove the previous log folder
!rm -rf ./tflogs/
# Parameters
lr = 0.0001
batch_size = 64
epochs = 2000

# Constructing CNN model
cnn2 = tf.keras.Sequential()
cnn2.add(tf.keras.layers.Conv1D(filters = 16, kernel_size = 3, input_shape = (128, 9)))
cnn2.add(tf.keras.layers.BatchNormalization())
cnn2.add(tf.keras.layers.Conv1D(filters = 32, kernel_size = 4))
cnn2.add(tf.keras.layers.BatchNormalization())
cnn2.add(tf.keras.layers.Conv1D(filters = 64, kernel_size = 5))
cnn2.add(tf.keras.layers.BatchNormalization())
cnn2.add(tf.keras.layers.ReLU())
cnn2.add(tf.keras.layers.GlobalAveragePooling1D())
cnn2.add(tf.keras.layers.Dense(units = 6, activation = tf.nn.softmax))

# Set up logging
logdir = 'tflogs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

# creating callback to save best model
callbacks = [
    tf.keras.callbacks.ModelCheckpoint("best_model_cnn2", save_best_only=True, monitor="val_accuracy",
                                       mode='max', verbose = 0),
    tf.keras.callbacks.TensorBoard(log_dir=logdir)
]

# Compiling
cnn2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = lr),
			loss='categorical_crossentropy', metrics=['accuracy'])
# Fitting
cnn2.fit(x_train_all, y_train1h, epochs = epochs, batch_size = batch_size, verbose = 0,
        validation_data=(x_val_all, y_val1h), callbacks=callbacks)
# recording best model configuration
net3_2 = tf.keras.models.load_model("best_model_cnn2")
# Training accuracy
print("Training loss and accuracy:", net3_2.evaluate(x_train_all, y_train1h))
# Validation accuracy
print("Validation loss and accuracy:", net3_2.evaluate(x_val_all, y_val1h))

In [ ]:
# Model from Part 3 test accuracy
print("Test loss and accuracy:", net3_2.evaluate(xtest_all, y_test1h))